In [2]:
import boto3
import os

# Initialize the Rekognition client
rekognition = boto3.client('rekognition')

# Create a collection
def create_collection(collection_id):
    try:
        response = rekognition.create_collection(CollectionId=collection_id)
        print(f"Collection {collection_id} created. ARN: {response['CollectionArn']}")
    except rekognition.exceptions.ResourceAlreadyExistsException:
        print(f"Collection {collection_id} already exists.")

# Index a face
def index_face(collection_id, image_path, employee_id):
    with open(image_path, 'rb') as image:
        response = rekognition.index_faces(
            CollectionId=collection_id,
            Image={'Bytes': image.read()},
            ExternalImageId=employee_id,
            DetectionAttributes=['ALL']
        )
    print(f"Face indexed for employee {employee_id}")
    return response['FaceRecords'][0]['Face']['FaceId']

# Recognize a face
def recognize_face(collection_id, image_path):
    with open(image_path, 'rb') as image:
        response = rekognition.search_faces_by_image(
            CollectionId=collection_id,
            Image={'Bytes': image.read()},
            MaxFaces=1,
            FaceMatchThreshold=95
        )
    
    if response['FaceMatches']:
        return response['FaceMatches'][0]['Face']['ExternalImageId']
    else:
        return None


In [3]:
import boto3
import os

def create_collection(collection_id):
    try:
        response = rekognition.create_collection(CollectionId=collection_id)
        print(f"Collection {collection_id} created. ARN: {response['CollectionArn']}")
    except rekognition.exceptions.ResourceAlreadyExistsException:
        print(f"Collection {collection_id} already exists.")

# Initialize the Rekognition client
rekognition = boto3.client('rekognition')
collection_id = "StudentFaces"
create_collection(collection_id)

Collection StudentFaces created. ARN: aws:rekognition:ap-south-1:961341519735:collection/StudentFaces


In [ ]:
student_folder = "student_photos"
for filename in os.listdir(student_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        employee_id = os.path.splitext(filename)[0]
        image_path = os.path.join(student_folder, filename)
        index_face(collection_id, image_path, employee_id)

In [ ]:


    # Index faces (add new employees)
    employee_folder = "student_photos"
    for filename in os.listdir(employee_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            employee_id = os.path.splitext(filename)[0]
            image_path = os.path.join(employee_folder, filename)
            index_face(collection_id, image_path, employee_id)

    # Recognize face (mark attendance)
    attendance_photo = "attendance_photo.jpg"
    recognized_employee = recognize_face(collection_id, attendance_photo)
    
    if recognized_employee:
        print(f"Attendance marked for employee: {recognized_employee}")
    else:
        print("No matching face found.")

In [2]:
slot_map = {
    "Monday": {
        "08:00 AM - 08:50 AM": "A1",
        "09:00 AM - 09:50 AM": ["F1", "A1"],
        "10:00 AM - 10:50 AM": "D1",
        "11:00 AM - 11:50 AM": "TB1",
        "12:00 PM - 12:50 PM": "TG1",
        "02:00 PM - 02:50 PM": "A2",
        "03:00 PM - 03:50 PM": "F2",
        "04:00 PM - 04:50 PM": "D2",
        "05:00 PM - 05:50 PM": "TB2",
        "06:00 PM - 06:50 PM": "TG2"
    },
    "Tuesday": {
        "08:00 AM - 08:50 AM": "B1",
        "09:00 AM - 09:50 AM": "G1",
        "10:00 AM - 10:50 AM": "E1",
        "11:00 AM - 11:50 AM": "TC1",
        "12:00 PM - 12:50 PM": "TAA1",
        "02:00 PM - 02:50 PM": "B2",
        "03:00 PM - 03:50 PM": "G2",
        "04:00 PM - 04:50 PM": "E2",
        "05:00 PM - 05:50 PM": "TC2",
        "06:00 PM - 06:50 PM": "TAA2"
    },
    "Wednesday": {
        "08:00 AM - 08:50 AM": "C1",
        "09:00 AM - 09:50 AM": "A1",
        "10:00 AM - 10:50 AM": "F1",
        "11:00 AM - 11:50 AM": "V2",
        "12:00 PM - 12:50 PM": "V2",
        "02:00 PM - 02:50 PM": "C2",
        "03:00 PM - 03:50 PM": "G2",
        "04:00 PM - 04:50 PM": "F2",
        "05:00 PM - 05:50 PM": "TD2",
        "06:00 PM - 06:50 PM": "TBB2"
    },
    "Thursday": {
        "08:00 AM - 08:50 AM": "D1",
        "09:00 AM - 09:50 AM": "B1",
        "10:00 AM - 10:50 AM": "C1",
        "11:00 AM - 11:50 AM": "TCC1",
        "12:00 PM - 12:50 PM": "TCC1",
        "02:00 PM - 02:50 PM": "D2",
        "03:00 PM - 03:50 PM": "A2",
        "04:00 PM - 04:50 PM": "B2",
        "05:00 PM - 05:50 PM": "TA2",
        "06:00 PM - 06:50 PM": "TCC2"
    },
    "Friday": {
        "08:00 AM - 08:50 AM": "E1",
        "09:00 AM - 09:50 AM": "C1",
        "10:00 AM - 10:50 AM": "B1",
        "11:00 AM - 11:50 AM": "TD1",
        "12:00 PM - 12:50 PM": "TD1",
        "02:00 PM - 02:50 PM": "E2",
        "03:00 PM - 03:50 PM": "D2",
        "04:00 PM - 04:50 PM": "F2",
        "05:00 PM - 05:50 PM": "TA2",
        "06:00 PM - 06:50 PM": "TDD2"
    }
}

def find_slot(day, time):
    if day in slot_map:
        for slot_time, slot in slot_map[day].items():
            start_time, end_time = slot_time.split(" - ")
            if start_time <= time <= end_time:
                return slot
    return None

# Example usage
day = "Monday"
time = "08:30 AM"
slot = find_slot(day, time)
print(f"The slot for {day} at {time} is {slot}")

The slot for Monday at 08:30 AM is A1


In [1]:
import networkx

In [ ]:
networkx.bfs_edges